In [1]:
from pathlib import Path
import sys, os
import numpy as np
from pprint import pprint
from copy import deepcopy

# notebook이 examples/selectiveRIP에서 실행 중일 때
repo_root = Path.cwd().resolve().parents[1]  # .../qubit-experiment
sys.path.insert(0, str(repo_root))

In [2]:

from laboneq.simple import *
from laboneq.dsl.device import DeviceSetup
from laboneq.dsl.calibration import Oscillator, SignalCalibration
from laboneq.dsl.enums import ModulationType
from example_helpers.generate_descriptor import generate_descriptor


descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=3,
    multiplex=True,
    number_multiplex=3,
    include_cr_lines=False,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="sibal_test",
    ip_address="192.168.0.83"
)




#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/QC6CH'


bus_ids = [f"b{i}" for i in range(3)]
bus_port = [4,5,6] #used 1,2,3 for qubit drive

for i, bus in zip(bus_port,bus_ids):
    setup.add_connections(
        setup.instruments[0].uid,
        # each bus uses its own drive:
        create_connection(
            to_signal=f"{bus}/drive",
            ports=f"SGCHANNELS/{i}/OUTPUT"
        ))

# Calibrate qubit drive/measure lines for oscillator phase increments
qubit_ids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
for qubit in qubit_ids:
    for line, frequency, mod_type in [
        ("drive", 5e9, ModulationType.HARDWARE),
        ("drive_ef", 6e9, ModulationType.HARDWARE),
        ("measure", 4e9, ModulationType.SOFTWARE),
    ]:
        logical_signal = setup.logical_signal_by_uid(f"{qubit}/{line}")
        oscillator = Oscillator(modulation_type=mod_type)
        logical_signal.calibration = SignalCalibration(
            local_oscillator=Oscillator(frequency=frequency),
            oscillator=oscillator,
        )
        if line == "measure":
            acquire_signal = setup.logical_signal_by_uid(f"{qubit}/acquire")
            acquire_signal.calibration = SignalCalibration(
                local_oscillator=Oscillator(frequency=frequency),
                oscillator=oscillator,
            )


In [3]:

from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
#workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


# Initial QPU

In [4]:
from qpu_types.fixed_transmon import FixedTransmonQubit, FixedTransmonQubitParameters
from qpu_types.bus_cavity import BusCavity, BusCavityParameters
from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from helper import load_qubit_parameters, save_qubit_parameters

qubit_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
bus_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("b")]

qubits = FixedTransmonQubit.from_device_setup(
    setup, qubit_uids=qubit_uids)
buses = BusCavity.from_device_setup(
    setup, qubit_uids=bus_uids)

qpu = QPU(quantum_elements={"qubits" : qubits, "bus" : buses}, quantum_operations=[FixedTransmonOperations, BusCavityOperations])


# from laboneq.simple import workflow
# folder_store = workflow.logbook.FolderStore("./experiment_store") 
# folder_store.activate()
# #folder_store.deactivate()
# #workflow.logbook.LoggingStore().activate()
# #workflow.logbook.LogbookStore().deactivate()


# SAVE QPU

From now on we save/load QPU elements not QuantumElements

In [ ]:
from laboneq.serializers import save, load, from_dict, from_json, to_dict, to_json
import time

t = time.localtime()
timestamp = time.strftime('%Y%m%d-%H%M', t)

filename = "save_test2"
save(qpu, filename=f"./qpu_parameters/{timestamp}_{filename}")



# LOAD QPU

Likewise

In [5]:
from datetime import datetime
def find_latest_json(folder_path):
    files = [f for f in os.listdir(folder_path)]
    timestamps = []
    for file in files:
        try:
            # Extract a timestamp assuming it's included in the filename
            timestamp_str = file.split('_', 1)[0]  # Assuming YYYYMMDDHHMMSS format
            timestamp = datetime.strptime(timestamp_str, '%Y%m%d-%H%M%S')
            timestamps.append((timestamp, file))
        except ValueError:
            print("fuck")
            continue  # Skip files that do not match the timestamp format

       # Find the most recent file
    if timestamps:
        latest_file = max(timestamps, key=lambda x: x[0])[1]
        return os.path.join(folder_path, latest_file)
    return None

qb_pars_file = find_latest_json("./qpu_parameters/")
print(f"LOADED: {qb_pars_file}")

from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
import laboneq.dsl.quantum.qpu as qpu_mod

class CombinedOperations(FixedTransmonOperations, BusCavityOperations):
    pass

qpu_mod.CombinedOperations = CombinedOperations

qpu = load(qb_pars_file)

buses = qpu.groups.bus
qubits = qpu.groups.qubits

LOADED: ./qpu_parameters/20260107-1548_save_test2


# Connection

In [6]:
session = Session(setup)
session.connect(do_emulation=True)

[2026.01.07 15:54:47.939] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/examples/selectiveRIP/laboneq_output/log
[2026.01.07 15:54:47.942] INFO    VERSION: laboneq 25.10.2
[2026.01.07 15:54:47.943] INFO    Connecting to data server at 192.168.0.83:8004
[2026.01.07 15:54:47.944] INFO    Connected to Zurich Instruments LabOne Data Server version 25.10.1.4 at 192.168.0.83:8004
[2026.01.07 15:54:47.945] INFO    Configuring the device setup
[2026.01.07 15:54:47.948] INFO    The device setup is configured


In [7]:
# # Q2 q0 -> q0
# qubits[0].parameters.drive_lo_frequency = 4.0e9
# qubits[0].parameters.drive_range = 10
# qubits[0].parameters.ge_drive_amplitude_pi = 0.6661475528508879
# qubits[0].parameters.ge_drive_amplitude_pi2 = 0.33101490643459497
# qubits[0].parameters.ge_drive_length = 64e-9

# qubits[0].parameters.readout_amplitude = 0.5
# qubits[0].parameters.readout_length = 1.2e-6
# qubits[0].parameters.readout_integration_delay = 400e-9
# qubits[0].parameters.readout_lo_frequency = 7.4e9
# qubits[0].parameters.readout_pulse = {
#             "function": "GaussianSquare",
#             "sigma": 0.2,
#             "risefall_sigma_ratio": 3.0
#         }
# qubits[0].parameters.readout_range_in = 0
# qubits[0].parameters.readout_range_out = 0

# qubits[0].parameters.resonance_frequency_ge = 4124420661.263548
# qubits[0].parameters.resonance_frequency_ef = 4.0e9
# qubits[0].parameters.readout_resonator_frequency = 7558567966.629468

# # Q3 q1 -> q1



# # bus mode 1
# buses[0].parameters.resonance_frequency_bus = 5.4947e9
# buses[0].parameters.rip_detunning = -13e6
# buses[0].parameters.drive_lo_frequency = 5.4e9
# # bus mode 2
# buses[1].parameters.resonance_frequency_bus = 5.9687e9
# buses[1].parameters.rip_detunning = -13e6
# buses[1].parameters.drive_lo_frequency = 6.4e9
# # bus mode 3
# buses[2].parameters.resonance_frequency_bus = 6.4203e9
# buses[2].parameters.rip_detunning = -13e6
# buses[2].parameters.drive_lo_frequency = 6.4e9


In [ ]:
buses[0].parameters.resonance_frequency_bus

# Calibration

## Ramsey 

In [ ]:
from laboneq_applications.experiments import ramsey


q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temp_pars.resonance_frequency_ge = 4.9528e9
temp_pars.readout_length = 1.2e-6
temp_pars.readout_integration_length =1.2e-6
temporary_parameters[q.uid] = temp_pars
#######################################################################
delays = np.linspace(0,10e-6,301)
detunings = 0.6e6,
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(False)
options.count(512)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options,
    temporary_parameters=temporary_parameters
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)

# X90

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars
#######################################################################
repetitions =np.arange(1,10)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(False)
options.use_cal_traces(True)
options.do_pca(False)
options.count(1024)

#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters=temporary_parameters,
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

# X180

In [10]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[0]
temporary_parameters = {}
temp_pars =deepcopy(q.parameters)
temporary_parameters[q.uid] = temp_pars

#######################################################################
repetitions =np.arange(1,14) # Due to short T1, 20   
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(False)
options.use_cal_traces(True)
#options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters=temporary_parameters,
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2026.01.07 15:57:18.067] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.01.07 15:57:18.068] INFO      Workflow 'experiment_workflow_x180': execution started at 2026-01-07          
[2026.01.07 15:57:18.068] INFO      06:57:18.067350Z                                                              
[2026.01.07 15:57:18.069] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.01.07 15:57:18.086] INFO    Task 'temporary_qpu': started at 2026-01-07 06:57:18.086151Z
[2026.01.07 15:57:18.087] INFO    Task 'temporary_qpu': ended at 2026-01-07 06:57:18.087409Z
[2026.01.07 15:57:18.090] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-01-07 
[2026.01.07 15:57:18.091] INFO    06:57:18.090571Z
[2026.01.07 15:57:18.092] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-01-07 06:57:18.092351Z
[2026.01.07 15:57:18.094] INFO    Task 'create_experiment': started at 2026-01-07 06:

# RB

In [9]:
#from laboneq_applications.experiments import 
from experiments import single_qubit_randomized_benchmarking
q = qubits[0]
# temporary_parameters = {}
# temp_pars =deepcopy(q.parameters)
# temporary_parameters[q.uid] = temp_pars

#############################################
options = single_qubit_randomized_benchmarking.experiment_workflow.options()
options.count(50)
options.transition("ge")


rb = single_qubit_randomized_benchmarking.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    length_cliffords=[1,5,10,20,50],
    variations=10,
    options=options,
)

rb_result = rb.run()

#######################################################################

[2026.01.07 15:55:35.556] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.01.07 15:55:35.556] INFO      Workflow 'single_qubit_randomized_benchmarking': execution started at         
[2026.01.07 15:55:35.557] INFO      2026-01-07 06:55:35.555626Z                                                   
[2026.01.07 15:55:35.557] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.01.07 15:55:35.582] INFO    Task 'get_gate_map': started at 2026-01-07 06:55:35.581854Z
[2026.01.07 15:55:35.583] INFO    Task 'get_gate_map': ended at 2026-01-07 06:55:35.582867Z
[2026.01.07 15:55:35.583] INFO    Task 'add_qasm_operations': started at 2026-01-07 06:55:35.583506Z
[2026.01.07 15:55:35.583] WARNING Type <class '__main__.CombinedOperations'> not supported by the serializer [name: add_qasm_operations.input.quantum_operations].
[2026.01.07 15:55:35.584] INFO    Task 'add_qasm_operations': ended at 2026-01-07 06:55:35

# RIP ECHO

In [ ]:
from experiments import rip_echo

#q = qubits[1]
#temporary_parameters = q.parameters.copy()

#######################################################################

#######################################################################
options = rip_echo.experiment_workflow.options()
options.update(False)

#print(workflow.show_fields(options))

###################################################################
bus_freq = qubits[2].parameters.resonance_frequency_ge - 13e6#45e6+26e6 
bus_amp = 0.45

bus2_freq = qubits[4].parameters.resonance_frequency_ge - 9e6  
bus2_amp = 0.0

rip_echo_controlg = rip_echo.experiment_workflow(
    session=session,
    qpu=qpu,
    ctrl=qubits[1],
    targ=qubits[0],
    bus=buses[0], #mode1 
    bus2=buss[4], #mode3 
    bus_frequency=bus_freq,
    bus_amplitude=bus_amp,
    bus2_frequency=bus2_freq,
    bus2_amplitude=bus2_amp,
    delays=np.linspace(80e-9, 2400e-9, 200),
    detunings=0,
    c_prep = "g",
    options=options 
)
rip_echo_controle = rip_echo.experiment_workflow(
    session=session,
    qpu=qpu,
    ctrl=qubits[1],
    targ=qubits[0],
    bus=qubits[2],
    bus2=qubits[4],
    bus_frequency=bus_freq, 
    bus_amplitude=bus_amp,
    bus2_frequency=bus2_freq,
    bus2_amplitude=bus2_amp,
    delays=np.linspace(80e-9, 2400e-9, 200),
    detunings=0,
    c_prep = "e",
    options=options 
)

rip_echo_result_control_g = rip_echo_controlg.run()
rip_echo_result_control_e = rip_echo_controle.run()
